# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Clusterwise PLS

In [ ]:
import clustering4ever.spark.clustering.clusterwise.{Clusterwise, ClusterwiseCore, PLS}
import scala.collection.mutable

import clustering4ever.spark.clustering.clusterwise.{Clusterwise, ClusterwiseCore, PLS}
import scala.collection.mutable


In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Clusterwise/simdata.csv

--2018-05-17 14:15:33--  http://www.clustering4ever.org/Datasets/Clusterwise/simdata.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61840 (60K) [text/csv]
Saving to: ‘/tmp/simdata.csv’

     0K .......... .......... .......... .......... .......... 82% 53.1K 0s
    50K ..........                                            100% 57.4M=0.9s

2018-05-17 14:15:35 (64.2 KB/s) - ‘/tmp/simdata.csv’ saved [61840/61840]


import sys.process._


In [ ]:
val dataXY = scala.io.Source.fromFile("/tmp/simdata.csv")
  .getLines
  .map(_.split(",").map(_.toDouble))
  .map( a => (a.take(10), a.takeRight(3)) )
  .zipWithIndex
  .map{ case(xy,idx) => (idx, xy) }
  .toSeq

dataXY: Seq[(Int, (Array[Double], Array[Double]))] = Stream((0,([D@5c0cb921,[D@3767aa2a)), ?)


## Run a full Clusterwise
### Parameters

In [ ]:
val init = 2
val nbCV = 4
val g = 2
val h = 2
val sizeBloc = 10
val kPred = 25
val nbLaunch = 1

init: Int = 2
nbCV: Int = 4
g: Int = 2
h: Int = 2
sizeBloc: Int = 10
kPred: Int = 25
nbLaunch: Int = 1


In [ ]:
val (sqrmseCalAndVal, _) = Clusterwise.run(sc, dataXY.take(200), g, h, nbCV, init, kPred)

sqrmseCalAndVal: Array[(Double, Double)] = Array((0.08851933065816746,10.541718991603423), (0.06831617620301471,6.5526965803161525), (0.08174661953527328,8.771838860689988), (0.08108279691003666,8.158809215689448))


## Run one Initialization of Clusterwise

In [ ]:
val (dsPerClass, predFitted, coIntercept, coXYcoef, critReg, mapsRegCrit, classedReg) = ClusterwiseCore.plsPerDot(dataXY.toArray, h, g)

dsPerClass: Array[Array[(Int, (Array[Double], Array[Double], Int))]] = Array(Array((7,(Array(6.685955917, 6.769605686, 5.342186991, 5.360624854, 5.205307736, 8.023073865, 8.646419433, 6.845253485, 7.939636587, 7.158224572),Array(6.857725732, 6.853841789, 6.627549623),0)), (10,(Array(3.004767498, 4.15832073, 6.432996097, 4.082413998, 3.831950687, 4.079528542, 3.470887844, 4.210125703, 3.577345581, 3.468564244),Array(4.88271408, 4.864995913, 5.308147136),0)), (18,(Array(6.839084225, 6.216818052, 2.76972574, 4.229632153, 4.569275396, 6.247105566, 6.015677646, 5.530191233, 5.151106324, 3.778566681),Array(5.139201214, 4.804874604, 4.914982244),0)), (19,(Array(2.722262812, 4.316130326, 4.603950916, 4.574134996, 4.502922088, 3.70118862, 3.862089795, 4.463502443, 3.609690828, 3.698167288),Array...

In [ ]:
// Need to form micro cluster into allGroupedData
//val (dsPerClass, predFitted, coIntercept, coXYcoef, critReg, mapsRegCrit, classedReg) = ClusterwiseCore.plsPerMicroClusters(dataXY.toArray, allGroupedData, h, g)

<console>:87: error: not enough arguments for method plsPerMicroClusters: (dsXYTrain: Array[(Int, (Array[Double], Array[Double]))], allGroupedData: scala.collection.immutable.HashMap[Int,Int], h: Int, g: Int, microClusterNumber: Int, nbMaxAttemps: Int, logOn: Boolean)(Array[Array[(Int, Array[(Int, Int, Array[(Int, Int, Array[Double], Array[Double])])])]], Array[Array[(Int, Array[Double])]], Array[Array[Double]], Array[Array[Double]], Array[Double], scala.collection.mutable.HashMap[Int,Double], Array[(Int, Int)]).
Unspecified value parameters g, microClusterNumber.
       val (dsPerClass, predFitted, coIntercept, coXYcoef, critReg, mapsRegCrit, classedReg) = ClusterwiseCore.plsPerMicroClusters(dataXY.toArray, h, g)
                                                                                                                                  ^


## Run one PLS regression

In [ ]:
val inputX = Array(mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => (id, x) }:_*))
val inputY = Array(mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => y }:_*))

PLS.runPLS(inputX, inputY, 0, 2)

inputX: Array[scala.collection.mutable.ArrayBuffer[(Int, Array[Double])]] = Array(ArrayBuffer((0,Array(4.058845733, 6.405834969, 4.943772245, 4.191393637, 5.691694039, 6.18161279, 6.182385889, 5.982450545, 6.139507172, 7.42310638)), (1,Array(5.102442757, 4.716437077, 4.692925255, 4.839302826, 4.02386069, 2.158591158, 2.901072862, 2.816463237, 3.501312504, 3.084936398)), (2,Array(4.766958169, 4.838583906, 6.485887384, 4.404467358, 5.211054103, 4.865929818, 5.278244008, 4.604447037, 4.98848043, 4.768183695)), (3,Array(6.138171718, 4.494158628, 3.410581785, 5.131981446, 5.551559701, 4.294727934, 5.917064075, 5.219849604, 5.133860522, 4.03092709)), (4,Array(5.375353406, 4.35623841, 4.635715716, 5.089608082, 3.678189181, 2.518132309, 4.754494191, 3.027216847, 3.052088737, 2.954102358)), (5,A...

(1656.019801616491,0.06979240528901356 0.06789620944460527 0.06930445371520205 
0.08067757263085021 0.07970448849559086 0.07974774446651722 
-0.07847738832689169 -0.08644739058771864 -0.0748970917978408 
0.08511456892838887 0.08420743469638495 0.08409775190792484 
0.09009642524949013 0.08958130237869746 0.08888651763257115 
0.14068789390390152 0.1435177674030875 0.13770796553300962 
0.13098330500261812 0.13316536912040206 0.1283447562823357 
8.524690943149948E-5 -0.003144389803075201 0.0010531588356896757 
0.11855232562338051 0.12009478549140792 0.11629398491749238 
0.10618779161310123 0.10710533264884624 0.10430424557462933 ,[D@3220f106,[Lscala.Tuple2;@64dbfa56)